In [1]:
from datetime import datetime
from modelhub import ModelHub
from bach import display_sql_as_markdown

In [2]:
modelhub = ModelHub(time_aggregation='%Y-%m-%d')

In [3]:
df = modelhub.get_objectiv_dataframe(start_date='2022-02-02')
df['feature_nice_name'] = df.location_stack.ls.nice_name

In [4]:
# construct steps column which is a list of strings 
df_steps = df.groupby('session_id')['feature_nice_name'].to_json_array().reset_index()
df_steps = df_steps.rename(columns={'feature_nice_name': 'steps'})
df_steps = df.merge(df_steps, on='session_id')[['session_id', 'steps']].drop_duplicates()
df_steps.to_pandas()

,session_id,steps
event_id,,
266f2183-b1c8-4445-ac09-1c681542ba6e,1,"[Root Location: home, Media Player: 2-minute-v..."
b9ab19f4-4ade-490e-9254-3ff35fe0cb42,2,"[Root Location: about, Root Location: home]"
dd6a9094-17fd-43d2-bb1d-4b62a6beec5d,3,"[Root Location: home, Media Player: 2-minute-v..."
027d6324-6e36-4ce1-8f40-1d5b6db5d4f4,4,[Expandable: The Project located at Root Locat...
0d2341dc-1ce2-485a-912a-3551b2b48aff,5,"[Root Location: home, Media Player: 2-minute-v..."
...,...,...
03104dbd-07ad-4517-b979-a40c5d845ce6,5349,"[Root Location: home, Media Player: 2-minute-v..."
132d1ba7-99b1-4b4c-b961-a9603fdf20ff,5350,[Link: tracking located at Root Location: trac...
b03d0d36-12a4-427d-afa2-5ac52b285adb,5351,[Link: star-us located at Root Location: home ...


## My custom lambda func

In [5]:
func_ngram = lambda data, n: [data[i: i + n] for i in range(len(data) - n + 1)]

func_ngram(['a', 'b', 'c', 'd', 'e', 'f'], 3)

[['a', 'b', 'c'], ['b', 'c', 'd'], ['c', 'd', 'e'], ['d', 'e', 'f']]

### For each row of `df_steps.steps` want to apply `func_ngram` func

In [6]:
df_steps_pandas = df_steps.to_pandas()
n_gram = 3 # this value will provide the user

df_steps_pandas['desired_col'] = df_steps_pandas['steps'].apply(lambda x: func_ngram(x, n_gram))

In [7]:
df_steps_pandas['steps'].iloc[0]

['Root Location: home',
 'Media Player: 2-minute-video located at Root Location: home',
 'Root Location: home',
 'Media Player: 2-minute-video located at Root Location: home',
 'Root Location: home',
 'Media Player: 2-minute-video located at Root Location: home',
 'Link: docs located at Root Location: home => Navigation: navbar-top',
 'Expandable: The Project located at Root Location: docs => Navigation: docs-sidebar',
 'Root Location: docs']

In [8]:
df_steps_pandas['desired_col'].iloc[0]

[['Root Location: home',
  'Media Player: 2-minute-video located at Root Location: home',
  'Root Location: home'],
 ['Media Player: 2-minute-video located at Root Location: home',
  'Root Location: home',
  'Media Player: 2-minute-video located at Root Location: home'],
 ['Root Location: home',
  'Media Player: 2-minute-video located at Root Location: home',
  'Root Location: home'],
 ['Media Player: 2-minute-video located at Root Location: home',
  'Root Location: home',
  'Media Player: 2-minute-video located at Root Location: home'],
 ['Root Location: home',
  'Media Player: 2-minute-video located at Root Location: home',
  'Link: docs located at Root Location: home => Navigation: navbar-top'],
 ['Media Player: 2-minute-video located at Root Location: home',
  'Link: docs located at Root Location: home => Navigation: navbar-top',
  'Expandable: The Project located at Root Location: docs => Navigation: docs-sidebar'],
 ['Link: docs located at Root Location: home => Navigation: navba